In [1]:
import os
import gc
import numpy as np
import json
import torch
from torch.utils.data import DataLoader
from torchinfo import summary

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from src.utils import *
from src.runner import run_config

from src.models import UNet, UNet2, UNet4, UNet8, UNetShallow, UNet2Shallow, UNet4Shallow, UNet8Shallow
from src.model_parts import SingleConv, DoubleConv, TripleConv

c:\Users\wittd\anaconda3\Lib\site-packages\pytools\persistent_dict.py:63: RecommendedHashNotFoundWarning: Unable to import recommended hash 'siphash24.siphash13', falling back to 'hashlib.sha256'. Run 'python3 -m pip install siphash24' to install the recommended hash.
  warn("Unable to import recommended hash 'siphash24.siphash13', "


Cupy implementation is not available. Make sure you have the right version of Cupy and CUDA installed.
Optional dependecy Dask_image is not installed. Implementations using it will be ignored.


c:\Users\wittd\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


In [2]:
config = {
'data_path': 'C:/7_felev/data',
    'save_path': 'C:/7_felev/szakdoga',
    # Each run name should be unique, even if the same model is used 
    # or it will overwrite the previous run
    'run_name': 'UNetSh4_tile10_len8_doubleconv',
    'project_name': 'Tile10',
    'wandb_logging': True,

    # Unet, UNet2, UNet4, UNet8, UNetShallow, Unet2Shallow, UNet4Shallow, UNet8Shallow
    'model': UNet4Shallow,
    'down_conv': DoubleConv,
    'up_conv': DoubleConv,
    'bilinear': False,

    'biosensor_length': 8,
    'input_scale': 1,
    # for the imput scaling
    'SRRF_mode': 'eSRRF', # 'eSRRF', 'SRRF', None
    'mask_scale': 4,
    
    'batch_size': 13,
    'epochs': 20,
    'learning_rate': 0.01,
    # K-fold cross validation
    'k': 5,

    'mask_type': bool,
    'augment': True,
    'normalize': True,
    'shuffle': True,
    
    'tiling': True,
    'tiling_ratio': 8,
    # tile size = (80 * input_scale) / tiling_ratio
    'overlap_rate': 0.2,

    'dilation': 0,
}

test_tiling(config)
run_config(config)

Tiling test
Bio tiles: torch.Size([81, 8, 10, 10]), Mask tiles: torch.Size([81, 40, 40])

Using device cuda
Datasets created!
Starting run 1...
UNet4Shallow


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: wdavid (wdave). Use `wandb login --relogin` to force relogin


Starting training:
        Epochs:          20
        Batch size:      13
        Learning rate:   0.01
        Training size:   104
        Validation size: 26
        Device:          cuda
        Mixed Precision: True
    


Validation Dice score: 0.2986704707145691, Detection rate: 0.0


Epoch 1/20: 100%|██████████| 104/104 [00:30<00:00,  3.45img/s, loss (batch)=1.09]


...

Epoch 20/20: 100%|██████████| 104/104 [00:25<00:00,  4.06img/s, loss (batch)=0.743]


Checkpoint 20 saved!


detection_rate,▁▇▆▆▇▆█▇▆▆▇▆▇▇▆▇▇█▇▇
epoch,▁▁▁▁▂▂▂▂▂▂▃▄▄▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇█████
learning rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇███
train loss,█▇▆▅▅▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▂▁▁▂▁▁▂▂▁▂▁▂▁▂▂▂
validation Dice,▅▁▆▇▇▇▇▇███▇████████
detection_rate,0.65736
epoch,20
learning rate,0.01
step,160
train loss,0.74253




Best epoch: 19


Test metrics: {
    "dice_score": 0.45926129817962646,
    "detection_rate": 0.6194046306504961,
    "avg_pred_cell_size": 1135.3712397650409,
    "std_pred_cell_size": 725.7389911951682,
    "avg_mask_cell_size": 580.0544377067255,
    "std_mask_cell_size": 525.6439429871493
}
Run 5 finished!
All runs finished!


In [3]:
config2 = config.copy()
config2['run_name'] = 'UNetSh4_tile10_len8_singleconv'
config2['down_conv'] = config['up_conv'] = SingleConv
test_tiling(config2)
run_config(config2)

Tiling test
Bio tiles: torch.Size([81, 8, 10, 10]), Mask tiles: torch.Size([81, 40, 40])

Using device cuda
Datasets created!
Starting run 1...
UNet4Shallow


Starting training:
        Epochs:          20
        Batch size:      13
        Learning rate:   0.01
        Training size:   104
        Validation size: 26
        Device:          cuda
        Mixed Precision: True
    


Validation Dice score: 0.2986704707145691, Detection rate: 0.0


Epoch 1/20: 100%|██████████| 104/104 [00:28<00:00,  3.68img/s, loss (batch)=1.04]


Checkpoint 1 saved!


...

Epoch 20/20: 100%|██████████| 104/104 [00:26<00:00,  3.97img/s, loss (batch)=0.734]


Checkpoint 20 saved!


detection_rate,▁▅█▃▃▄▅▄▄▃▄▅▅▄▄▄▄▄▄▄
epoch,▁▁▁▁▂▂▃▃▃▃▃▃▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██████
learning rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
step,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇██
train loss,█▃▃▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▂▁▂▁▁▁▁▁▁▂▁▁▁▁▁▂▁
validation Dice,▁▁▄▆▅▇▄▅▇▆▇▇▇█▇█▇███
detection_rate,0.76518
epoch,20
learning rate,0.01
step,160
train loss,0.73387




Best epoch: 19


Test metrics: {
    "dice_score": 0.45664650201797485,
    "detection_rate": 0.6562293274531422,
    "avg_pred_cell_size": 1179.7898815524193,
    "std_pred_cell_size": 802.6464109422374,
    "avg_mask_cell_size": 580.0544377067255,
    "std_mask_cell_size": 525.6439429871493
}
Run 5 finished!
All runs finished!


In [4]:
config = {
    'data_path': 'C:/7_felev/data',
    'save_path': 'C:/7_felev/szakdoga',
    # Each run name should be unique, even if the same model is used 
    # or it will overwrite the previous run
    'run_name': 'UNetSh8_tile10_len8_doubleconv',
    'project_name': 'Tile10',
    'wandb_logging': True,

    # Unet, UNet2, UNet4, UNet8, UNetShallow, Unet2Shallow, UNet4Shallow, UNet8Shallow
    'model': UNet8Shallow,
    'down_conv': DoubleConv,
    'up_conv': DoubleConv,
    'bilinear': False,

    'biosensor_length': 8,
    'input_scale': 1,
    # for the imput scaling
    'SRRF_mode': 'eSRRF', # 'eSRRF', 'SRRF', None
    'mask_scale': 8,
    
    'batch_size': 13,
    'epochs': 20,
    'learning_rate': 0.01,
    # K-fold cross validation
    'k': 5,

    'mask_type': bool,
    'augment': True,
    'normalize': True,
    'shuffle': True,
    
    'tiling': True,
    'tiling_ratio': 8,
    # tile size = (80 * input_scale) / tiling_ratio
    'overlap_rate': 0.2,

    'dilation': 0,
}

test_tiling(config)
run_config(config)

Tiling test
Bio tiles: torch.Size([81, 8, 10, 10]), Mask tiles: torch.Size([81, 80, 80])

Using device cuda
Datasets created!
Starting run 1...
UNet8Shallow


Starting training:
        Epochs:          20
        Batch size:      13
        Learning rate:   0.01
        Training size:   104
        Validation size: 26
        Device:          cuda
        Mixed Precision: True
    


Validation Dice score: 0.2905982732772827, Detection rate: 0.0


Epoch 1/20: 100%|██████████| 104/104 [00:29<00:00,  3.52img/s, loss (batch)=1.13]


Checkpoint 1 saved!


...

Epoch 20/20: 100%|██████████| 104/104 [00:26<00:00,  3.95img/s, loss (batch)=0.689]


Checkpoint 20 saved!


detection_rate,▁▁▁█▅▆▅▆▇▅▇▆▆▇▇▇▆▇▇▇
epoch,▁▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇██████
learning rate,█████████████▁▁▁▁▁▁▁
step,▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇███
train loss,█▇▆▆▅▄▃▃▃▄▃▂▂▃▂▂▂▂▃▂▂▂▂▂▂▂▂▂▂▃▂▂▂▂▂▂▂▁▂▁
validation Dice,▁▁▁▁▆▇▆▇▆▆▇▇▇▄▇█████
detection_rate,0.70776
epoch,20
learning rate,0.001
step,160
train loss,0.6893




Best epoch: 19


Test metrics: {
    "dice_score": 0.4584290385246277,
    "detection_rate": 0.6051897919794124,
    "avg_pred_cell_size": 1030.346108699504,
    "std_pred_cell_size": 754.0128121859469,
    "avg_mask_cell_size": 563.9999229305168,
    "std_mask_cell_size": 518.2801586694156
}
Run 5 finished!
All runs finished!


In [5]:
config2 = config.copy()
config2['run_name'] = 'UNetSh8_tile10_len8_singleconv'
config2['down_conv'] = config['up_conv'] = SingleConv
test_tiling(config2)
run_config(config2)

Tiling test
Bio tiles: torch.Size([81, 8, 10, 10]), Mask tiles: torch.Size([81, 80, 80])

Using device cuda
Datasets created!
Starting run 1...
UNet8Shallow


Starting training:
        Epochs:          20
        Batch size:      13
        Learning rate:   0.01
        Training size:   104
        Validation size: 26
        Device:          cuda
        Mixed Precision: True
    


Validation Dice score: 0.2905982732772827, Detection rate: 0.0


Epoch 1/20: 100%|██████████| 104/104 [00:29<00:00,  3.55img/s, loss (batch)=1.04]


Checkpoint 1 saved!


...

Epoch 20/20: 100%|██████████| 104/104 [00:28<00:00,  3.64img/s, loss (batch)=0.777]


Checkpoint 20 saved!


detection_rate,▁▁▁▁▁▁▁▁▁▁▇█████████
epoch,▁▁▁▂▂▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▇▇▇████
learning rate,██████▁▁▁▁▁▁▁▁▁▁▁▁▁▁
step,▁▁▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇█████
train loss,█▅▄▃▃▃▂▂▂▂▂▂▁▁▂▂▁▁▁▂▁▁▁▁▂▁▁▁▁▂▁▂▂▁▁▁▁▁▁▁
validation Dice,▁▁▁▁▁▁▁▁▁▁▇▇████████
detection_rate,0.72352
epoch,20
learning rate,0.001
step,160
train loss,0.77725




Best epoch: 19


Test metrics: {
    "dice_score": 0.4499771296977997,
    "detection_rate": 0.6302809350203732,
    "avg_pred_cell_size": 1227.5280973545423,
    "std_pred_cell_size": 909.9474324041925,
    "avg_mask_cell_size": 563.9999229305168,
    "std_mask_cell_size": 518.2801586694156
}
Run 5 finished!
All runs finished!
